<a href="https://colab.research.google.com/github/datarobot-community/mlops-examples/blob/master/Custom%20Model%20Examples/Insurance%20Pricing/Main_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DRUM Wild Mode

DRUM Wild Mode, formally known as Unstructured Mode, allows user to to pass and return almost anything.  No validation of data occurs and it is your responsibility to verify correctness.

In this example, we will host an LGMB Regressor and for every request, we will return predictions and shap values for said predictions.  


In [ ]:
!git clone https://github.com/datarobot-community/mlops-examples.git

In [3]:
!pip install -r /content/mlops-examples/custom_inference/python/insurance_pricing/requirements.txt -q

In [ ]:
!pip install datarobot-drum

     |████████████████████████████████| 8.7MB 951kB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 788kB 39.3MB/s 
     |████████████████████████████████| 276kB 30.0MB/s 
     |████████████████████████████████| 204kB 43.8MB/s 
     |████████████████████████████████| 153kB 40.3MB/s 
     |████████████████████████████████| 112kB 44.5MB/s 
     |████████████████████████████████| 808kB 46.6MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 204kB 39.9MB/s 
     |████████████████████████████████| 552kB 43.3MB/s 
  Created wheel for strictyaml: filename=strictyaml-1.0.6-cp36-none-any.whl size=25273 sha256=046b9bf311cd19405a1260beb9aa9c6cefde7c14380b0b5f97efb282b2fe6906
  Stored in directory: /root/.cache/pip/wheels/b8/db/4d/deb821b0efddef300d0788a44daa2fd5cefd886789ab8e0027
  Created wheel for progress: filename=progress-1.5-cp36-none-any.whl size=8074 sha256=5628e24186f68b1cdb903c04c00c5d57c9ac

In [4]:
start_server = ['drum',
 'server',
 '--code-dir',
 '/content/mlops-examples/custom_inference/python/insurance_pricing',
 '--target-type',
 'unstructured',
 '--address',
 'localhost:6789',
 '--verbose']

In [6]:
import io
import requests
import avro.io
import avro
from avro.datafile import DataFileReader
import subprocess
import pandas as pd

In [ ]:
server = subprocess.Popen(start_server, stdout=subprocess.PIPE)

In [ ]:
requests.request("GET", "http://localhost:6789/").content

b'{"message":"OK"}\n'

In [ ]:
csv_in_bytes = open("/content/mlops-examples/custom_inference/python/insurance_pricing/data/loss_cost_short.csvrb").read()

In [ ]:
import requests

url = "http://localhost:6789/predictUnstructured/?ret_mode=binary"

payload = csv_in_bytes
headers = {
  'Content-Type': 'application/octet-stream'
}

response = requests.request("POST", url, headers=headers, data = payload)

return_bytes = response.content

In [ ]:
schema = avro.schema.parse(open("/content/mlops-examples/custom_inference/python/insurance_pricing/schema.avscrb").read())
bytes_reader = io.BytesIO(return_bytes)
datum_reader = avro.io.DatumReader(schema)
returned_data = DataFileReader(bytes_reader, avro.io.DatumReader())
predictions = []  
for pred in returned_data:
    temp = pred["shap_values"]
    temp["prediction"] = pred["prediction"]
    predictions.append(temp)
pd.DataFrame(predictions).iloc[0]

SHAP_CustomerTenure                    -0.009229
SHAP_DriverAge                         -0.067060
SHAP_EngineCapacity                    -0.126928
SHAP_NumberOfDrivers                   -0.210533
SHAP_VehicleAge                        -0.002556
SHAP_VehicleCostNew                    -0.007075
SHAP_Zipcode_Aged_18_24                -0.021351
SHAP_Zipcode_Aged_25_29                -0.006317
SHAP_Zipcode_Aged_30_39                 0.009671
SHAP_Zipcode_Aged_40_44                -0.003087
SHAP_Zipcode_Aged_45_49                -0.286157
SHAP_Zipcode_Aged_50_59                -0.050674
SHAP_Zipcode_Aged_60                    0.100407
SHAP_Zipcode_CommuteViaCar              0.497439
SHAP_Zipcode_PersonsPerHousehold        0.001143
SHAP_Zipcode_VehiclesPerHousehold       0.030764
SHAP_Zipcode_annualMileage             -0.006449
SHAP_DistributionChannel               -0.008335
SHAP_VehicleModel                       0.068915
SHAP_Zipcode                           -0.001565
SHAP_VehicleMake    